In [ ]:
### SETUP

import os, gc

from time import time
from datetime import timedelta

import numpy as np
import statsmodels.api as sm

import pandas as pd
pd.options.mode.chained_assignment = None

import warnings
warnings.simplefilter(action = 'ignore', category = pd.errors.PerformanceWarning)

data_dir = os.getcwd() + '/data/'

In [ ]:
### DEFINE GROUP MAPS

race_cats = {
    1: 'w', # white
    2: 'b', # Black
    3: 'h', # Hispanic
}

sex_cats = {
    1: 'm', # male
    2: 'f', # female
}

age_cats = {
    1: 'a1', # 16-24
    2: 'a2', # 25-34
    3: 'a3', # 35-44
    4: 'a4', # 45-54
    5: 'a5', # 55-64
    6: 'a6', # 65+
}

educ_cats = {
    1: 'e1', # less than HS
    2: 'e2', # HS diploma
    3: 'e3', # some college
    4: 'e4', # col and higher
}

occ_cats = {
    0: 'miss',
    1: 'prof',
    2: 'tech',
    3: 'serv',
    4: 'farm',
    5: 'prod',
    6: 'oper',
}

ind_cats = {
    0: 'miss',
    1: 'agri',
    2: 'mine',
    3: 'cnst',
    4: 'manu',
    5: 'trns',
    6: 'whol',
    7: 'retl',
    8: 'fire',
    9: 'busi',
    10: 'pers',
    11: 'entr',
    12: 'prof',
    13: 'publ',
}

In [ ]:
### LOAD DATA AND SET REGRESSION FEATURES

est_df = pd.read_pickle(data_dir + '/cps_final.pkl')

outcome = 'unemployed'
macro_var = 'sur_sa_1y_avg'

age_dummies = ['age_cat_' + age_cats[i] for i in sorted(est_df['age_cat'].unique()) if i != 3]
educ_dummies = ['educ_cat_' + educ_cats[i] for i in sorted(est_df['educ_cat'].unique()) if i != 2]

occ_dummies = ['occ_cat_' + occ_cats[i] for i in sorted(est_df['occ_cat'].unique()) if i >= 2]
ind_dummies = ['ind_cat_' + ind_cats[i] for i in sorted(est_df['ind_cat'].unique()) if i != 0]

state_dummies = ['state_' + i for i in sorted(est_df['state'].unique()) if i != 'ca']
month_dummies = ['month_' + str(i) for i in sorted(est_df['month'].unique()) if i != 1]

addl_dummies = ['married', 'veteran', 'urban']
main_dummies = age_dummies + educ_dummies + occ_dummies + ind_dummies + addl_dummies

features = [macro_var] + main_dummies + [macro_var + '_' + i for i in main_dummies] + state_dummies + month_dummies + ['cons']
reg_vars = [outcome] + features + ['weight']

del est_df
gc.collect()

In [ ]:
### COLLECT OAXACA-BLINDER LPM ESTIMATES

t0 = time()

labels = []
spl_sizes = {}
unit_cnts = {}

wls_coefs = {}
wt_means = {}
wt_marg_effs = {}

for race in list(race_cats.keys()):
    for sex in list(sex_cats.keys()):
        t1 = time()
        
        label = outcome + '_' + macro_var + '_' + race_cats[race] + '_' + sex_cats[sex]
        labels.append(label)
        
        est_df = pd.read_pickle(data_dir + '/cps_final.pkl')
        est_df = est_df[(est_df['empl_cat'] != 3) & (est_df['race_cat'] == race) & (est_df['sex_cat'] == sex)]
        
        est_df.to_pickle(data_dir + '/cps_temp.pkl')
        est_df = pd.read_pickle(data_dir + '/cps_temp.pkl')
        
        est_df['weight'] = est_df['wtfinl'] * est_df['wtfinl'].count() / est_df['wtfinl'].sum()
        
        est_df['cons'] = 1
        est_df['cons'] = est_df['cons'].astype(np.int8)
        
        for i in sorted(est_df['age_cat'].unique()):
            est_df['age_cat_' + age_cats[i]] = np.where(est_df['age_cat'] == i, 1, 0).astype(np.int8)
        
        for i in sorted(est_df['educ_cat'].unique()):
            est_df['educ_cat_' + educ_cats[i]] = np.where(est_df['educ_cat'] == i, 1, 0).astype(np.int8)
        
        for i in sorted(est_df['occ_cat'].unique()):
            est_df['occ_cat_' + occ_cats[i]] = np.where(est_df['occ_cat'] == i, 1, 0).astype(np.int8)
        
        for i in sorted(est_df['ind_cat'].unique()):
            est_df['ind_cat_' + ind_cats[i]] = np.where(est_df['ind_cat'] == i, 1, 0).astype(np.int8)
        
        est_df.to_pickle(data_dir + '/cps_temp.pkl')
        est_df = pd.read_pickle(data_dir + '/cps_temp.pkl')
        
        for i in main_dummies:
            est_df[macro_var + '_' + i] = est_df[macro_var] * est_df[i]
        
        est_df.to_pickle(data_dir + '/cps_temp.pkl')
        est_df = pd.read_pickle(data_dir + '/cps_temp.pkl')
        
        for i in sorted(est_df['state'].unique()):
            est_df['state_' + i] = np.where(est_df['state'] == i, 1, 0).astype(np.int8)
        
        for i in sorted(est_df['month'].unique()):
            est_df['month_' + str(i)] = np.where(est_df['month'] == i, 1, 0).astype(np.int8)
        
        est_df[reg_vars].to_pickle(data_dir + '/cps_temp.pkl')
        est_df = pd.read_pickle(data_dir + '/cps_temp.pkl')
        
        spl_sizes[label] = int(est_df.shape[0])
        
        wls_betas = {}
        
        try:
            wls_model = sm.WLS(endog = est_df[outcome], exog = est_df[features], weights = est_df['weight']).fit()
            
            for i in features:
                wls_betas[i] = float(wls_model.params[i])
            
            est_df[outcome + '_fit'] = sum([est_df[i] * float(wls_model.params[i]) for i in features])
            unit_cnts[label] = int(est_df[outcome + '_fit'][(est_df[outcome + '_fit'] >= 0) & (est_df[outcome + '_fit'] <= 1)].count())
        except:
            for i in features:
                wls_betas[i] = 0.0
            
            unit_cnts[label] = 0
        
        wls_coefs[label] = wls_betas
        
        wt_avgs = {}
        wt_mes = {}
        
        for i in features:
            wt_avgs[i] = float((est_df[i] * est_df['weight']).sum() / spl_sizes[label])
            
            if i == macro_var:
                wt_mes[i] = wls_betas[i]
            elif i in main_dummies:
                wt_mes[i] = wt_avgs[i] * wls_betas[macro_var + '_' + i]
            else:
                wt_mes[i] = float('nan')
        
        wt_means[label] = wt_avgs
        wt_marg_effs[label] = wt_mes
        
        del est_df
        gc.collect()
        
        print(label + ' time: ' + str(timedelta(seconds = round(time() - t1))))

print('total time:', timedelta(seconds = round(time() - t0)))

In [ ]:
### COLLECT OAXACA-BLINDER LPM GAP ESTIMATES

coef_expl = {}
gaps = {}

for race in ['b', 'h']:
    for sex in ['m', 'f']:
        lbl = outcome + '_' + macro_var + '_' + race + '_' + sex
        w_lbl = outcome + '_' + macro_var + '_w_' + sex
        
        coef_expl[lbl] = {}
        gaps[lbl] = {}
        
        for f in features:
            if f in main_dummies:
                coef_expl[lbl][f] = (wt_means[lbl][f] - wt_means[w_lbl][f]) * wls_coefs[w_lbl][macro_var + '_' + f]
            else:
                coef_expl[lbl][f] = float('nan')
        
        gaps[lbl]['total'] = float(np.nansum(list(wt_marg_effs[lbl].values())) - np.nansum(list(wt_marg_effs[w_lbl].values())))
        gaps[lbl]['expl'] = float(np.nansum(list(coef_expl[lbl].values())))
        gaps[lbl]['unexpl'] = gaps[lbl]['total'] - gaps[lbl]['expl']
        
        gaps[lbl]['age'] = sum([coef_expl[lbl][i] for i in coef_expl[lbl] if i in age_dummies])
        gaps[lbl]['educ'] = sum([coef_expl[lbl][i] for i in coef_expl[lbl] if i in educ_dummies])
        gaps[lbl]['occ'] = sum([coef_expl[lbl][i] for i in coef_expl[lbl] if i in occ_dummies])
        gaps[lbl]['ind'] = sum([coef_expl[lbl][i] for i in coef_expl[lbl] if i in ind_dummies])
        gaps[lbl]['other'] = sum([coef_expl[lbl][i] for i in coef_expl[lbl] if i in addl_dummies])

In [ ]:
### PRINT OAXACA-BLINDER LPM ESTIMATES IN LATEX FORMAT

# helper function
def helper(dict_map, lbls):
    for lbl in lbls:
        rep = [lbl]
        
        for grp in [lbl for lbl in labels if 'w' not in lbl]:
            rep.append(round(100 * dict_map[grp][lbl], 4))
            rep.append(round(100 * dict_map[grp][lbl] / gaps[grp]['total'], 2))
        
        print(*rep, sep = ' & ')

# print gaps table
helper(gaps, ['total', 'expl', 'age'])
helper(coef_expl, age_dummies)
helper(gaps, ['educ'])
helper(coef_expl, educ_dummies)
helper(gaps, ['occ'])
helper(coef_expl, occ_dummies)
helper(gaps, ['ind'])
helper(coef_expl, ind_dummies)
helper(gaps, ['other'])
helper(coef_expl, addl_dummies)
helper(gaps, ['unexpl'])

In [ ]:
### PRINT OAXACA-BLINDER LPM UNIT/OBS COUNTS IN LATEX FORMAT

for lbl in labels:
    print(lbl + ' & ' + '{:,}'.format(spl_sizes[lbl]) + ' & ' + '{:,}'.format(unit_cnts[lbl]) + ' & ' + str(round(100 * unit_cnts[lbl] / spl_sizes[lbl], 2)) + ' \\\ ')

In [ ]:
### SAVE OAXACA-BLINDER LPM ESTIMATES

ob_df = pd.DataFrame(data = {'param': features})

for lbl in labels:
    ob_df['coef_' + lbl] = list(wls_coefs[lbl].values())
    ob_df['mean_' + lbl] = list(wt_means[lbl].values())
    ob_df['marg_eff_' + lbl] = list(wt_marg_effs[lbl].values())

for lbl in [l for l in labels if 'w' not in l]:
    ob_df['explained_' + lbl] = list(coef_expl[lbl].values())

ob_df.to_csv(os.getcwd() + '/results/ob_lpm_estimates.csv', index = False)